In [1]:
#Load Documents
import csv
import string
import sys

csv.field_size_limit(sys.maxsize)

train_raw_neg = []
train_raw_pos = []

with open('data/yelp/sentiment.train.0', 'rb') as fp:
    train_raw_neg = fp.readlines()
    
with open('data/yelp/sentiment.train.1', 'rb') as fp:
    train_raw_pos = fp.readlines()



In [2]:
neg_sent_cnt = len(train_raw_neg)
pos_sent_cnt = len(train_raw_pos)
print neg_sent_cnt
print pos_sent_cnt

177218
266041


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re


stop_word_list = list(stopwords.words('english'))

def tokenize_std(content, n_gram):
    content = content.lower()
    content = content.replace("\\n", " ")
    words = content.split()
    tokens = []
    for i in range(n_gram):
        for j in range(len(words)-i):
            tokens.append("_".join(words[j:j+i+1]))
    
    tokens = [w for w in tokens if not w in stop_word_list]
    return tokens


In [5]:
train_token_neg = []
train_token_pos = []

for s in train_raw_neg:
    train_token_neg.append(tokenize_std(s, 2))

for s in train_raw_pos:
    train_token_pos.append(tokenize_std(s, 2))
    

In [6]:
print train_token_neg[0:5]
print train_token_pos[0:5]

[['sadly', 'mistaken', '.', 'i_was', 'was_sadly', 'sadly_mistaken', 'mistaken_.'], ['hoagies', ',', 'italian', 'general', 'run', 'mill', '.', 'so_on', 'on_to', 'to_the', 'the_hoagies', 'hoagies_,', ',_the', 'the_italian', 'italian_is', 'is_general', 'general_run', 'run_of', 'of_the', 'the_mill', 'mill_.'], ['minimal', 'meat', 'ton', 'shredded', 'lettuce', '.', 'minimal_meat', 'meat_and', 'and_a', 'a_ton', 'ton_of', 'of_shredded', 'shredded_lettuce', 'lettuce_.'], ['nothing', 'really', 'special', '&', 'worthy', '$', '_num_', 'price', 'tag', '.', 'nothing_really', 'really_special', 'special_&', '&_not', 'not_worthy', 'worthy_of', 'of_the', 'the_$', '$__num_', '_num__price', 'price_tag', 'tag_.'], ['second', ',', 'steak', 'hoagie', ',', 'atrocious', '.', 'second_,', ',_the', 'the_steak', 'steak_hoagie', 'hoagie_,', ',_it', 'it_is', 'is_atrocious', 'atrocious_.']]
[['excellent', 'food', '.', 'excellent_food', 'food_.'], ['superb', 'customer', 'service', '.', 'superb_customer', 'customer_se

In [7]:
from nltk import ngrams
from collections import Counter

unigram_neg = Counter([])
unigram_pos = Counter([])
unigram_all = Counter([])

bigram_neg = Counter([])
bigram_pos = Counter([])
bigram_all = Counter([])


for s in train_token_neg:
    for w in s:
        bigram_neg[w] += 1
        bigram_all[w] += 1
    
for s in train_token_pos:
    for w in s:
        bigram_pos[w] += 1
        bigram_all[w] += 1


In [8]:
vocab_list = [w for w in bigram_all if (bigram_all[w] > 72)]
vocab_dict = {}
idx = 0
for w in vocab_list:
    vocab_dict[w] = idx
    idx += 1

with open('vocab_list.csv', 'wb') as csvfile:
    model_writer = csv.writer(csvfile)
    model_writer.writerow(vocab_list)    

print len(vocab_list)

8783


In [ ]:
import numpy as np


alpha = 1.0
d = len(vocab_list)
p, q = p, q = np.ones(d) * alpha , np.ones(d) * alpha

for w in vocab_list:
    p[vocab_dict[w]] += bigram_pos[w]
    q[vocab_dict[w]] += bigram_neg[w]

p /= abs(p).sum()
q /= abs(q).sum()
r = np.log(p/q)
    

In [ ]:
X_all = []
Y_all = []
for s in train_token_neg:
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]

    X_all.append(vec)
    Y_all.append(-1)

for s in train_token_pos:
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]

    X_all.append(vec)
    Y_all.append(1)


In [1]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit


X_train, X_test, Y_train, Y_test = train_test_split(X_all[0:100000]+X_all[300000:400000], Y_all[0:100000]+Y_all[300000:400000], test_size = 0.2)


NameError: name 'X_all' is not defined

In [21]:
from sklearn.svm import LinearSVC
#linear NBSVM model
clf_svc = LinearSVC(penalty='l2', C=0.1)
clf_svc.fit(X_train, Y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [22]:
import pickle
with open('classifier/clf_svc.pickle', 'wb') as f:
    pickle.dump(clf_svc, f)
    
with open('classifier/clf_svc.pickle', 'rb') as f:
    clf_svc_o = pickle.load(f)

print "Acurracy: %f" %clf_svc_o.score(X_test, Y_test)

Acurracy: 0.959156


In [23]:
from sklearn.linear_model import LogisticRegression
#NB LR model
clf_lr = LogisticRegression(C=1.0, penalty='l2', tol=0.001)
clf_lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False)

In [24]:
with open('classifier/clf_lr.pickle', 'wb') as f:
    pickle.dump(clf_lr, f)
    
with open('classifier/clf_lr.pickle', 'rb') as f:
    clf_lr_o = pickle.load(f)

print "Acurracy: %f" % clf_lr_o.score(X_test, Y_test)

Acurracy: 0.959375


In [17]:
print r

[ 0.6766152   1.00147899  0.81081579 ... -0.76472057  0.91358675
 -2.81290957]


In [44]:
#MNB model
cnt_true = 0.0
cnt_all = len(X_all)
for i in range(len(X_all)):
    log_prob = np.sum(X_all[i])
    if((log_prob > 0 and Y_all[i] == 1) or (log_prob < 0 and Y_all[i] == -1)):
        cnt_true += 1
        
print "Accuracy: %f" % (cnt_true/cnt_all)
        
        
def eval_sent_MNB(s):
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]
            
    return [np.sum(vec)]


    

Accuracy: 0.937251


In [25]:
def eval_sent_svm(clf, s):
    
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]
        
    #print clf.predict([vec])
    return clf.decision_function([vec])

def eval_sent_lr(clf, s):
    
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]
            
    return clf.predict_proba([vec])

In [98]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or